## Language Servers

`jupyter-lsp` does not come with any Language Servers! However, we will try to
use them if they _are_ installed and we know about them. For the language
servers in the tables below, use one of the suggested package managers to
install them: these implementations are tested to work with `jupyter-lsp`.

> _You can disable this feature by configuring_ >
> _[autodetect](./Configuring.ipynb#autodetect)_

If you do not see a language you would like, but can find it one of these lists:

- the [official list][lsp-implementations] of language servers
- a [community-curated list][langserver] of language servers

...you might be able to add it
[via configuration](./Configuring.ipynb#language_servers) or
[build your own spec](./Contributing.ipynb#Specs) for the server in question.

[language-server]:
  https://microsoft.github.io/language-server-protocol/specification
[langserver]: https://langserver.org
[lsp-implementations]:
  https://microsoft.github.io/language-server-protocol/implementors/servers

In [ ]:
import pathlib

import IPython
from jinja2 import Template

from jupyter_lsp import LanguageServerManager

In [ ]:
mgr = LanguageServerManager(extra_node_roots=[str(pathlib.Path.cwd().parent)])

In [ ]:
mgr.init_language_servers()

In [ ]:
def lang_server_table(specs):
    return IPython.display.HTML(
        Template(
            """
        <table class="langservers">
            <thead>
                <tr>
                    <th>Languages</th>
                    <th>Implementation</th>
                    <th>Installation</th>
                </tr>
            </thead>
            <tbody>
                {% for key, spec in specs.items() %}
                <tr>
                    <th>
                        {% for lang in spec.languages %}
                            <a name="language-{{lang}}"/>{{ lang }}<br/>
                        {% endfor %}
                    </th>
                    <td>
                        <a href="{{spec.urls.home}}">{{key}}</a>
                    </td>
                    <td>
                        <ul>
                        {% for pkgmgr, inst in spec.install.items() %}
                        <li>{{pkgmgr}}: <code>{{ inst }}</code></li>
                        {% endfor %}
                        </ul>
                    </td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        """
        ).render(specs=specs)
    )

### Notebook-Optimized Language Servers

These servers have well-tested support for notebooks and file editors.

In [ ]:
nb_langs = ["pyls", "r-languageserver"]
lang_server_table(
    {key: spec for key, spec in mgr.language_servers.items() if key in nb_langs}
)

### NodeJS-based Language Servers

These servers have mostly been tested with file editors.

In [ ]:
lang_server_table(
    {
        key: spec
        for key, spec in mgr.language_servers.items()
        if "npm" in spec["install"]
    }
)

#### Example: Getting All the NodeJS-based Language Servers

A number of language servers are built on the
[reference implementation](https://github.com/microsoft/vscode-languageserver-node),
powered by NodeJS. The most reliable place to install these is in a
`node_modules` in the directory where you launch `jupyter lab`.

For example, to install all the servers which are tested as part of
`jupyterlab-lsp`:

```bash
jlpm add --dev \
    bash-language-server \
    vscode-css-languageserver-bin \
    dockerfile-language-server-nodejs \
    vscode-html-languageserver-bin \
    javascript-typescript-langserver \
    vscode-json-languageserver-bin \
    yaml-language-server
```

This will create create (or add to):

- `package.json` (check this in!)
- `yarn.lock` (check this in!)
- `node_modules/` (add to your VCS ignore file)

If you wish to install these someplace else, you may need to specify where you
install them with [extra_node_roots](./Configuring.ipynb#extra_node_roots).

In [ ]:
%%html
<style>
    table.langservers thead th {
        text-align: center !important;
    }

    table.langservers tbody td,
    table.langservers tbody th {
        text-align: left !important;
    }
</style>